## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Cervo,43.6702,-7.4101,54.68,94,97,9.04,ES,light rain
1,1,Sinop,41.7711,34.8709,56.10,51,100,11.52,TR,overcast clouds
2,2,Itarema,-2.9248,-39.9167,77.70,88,67,5.77,BR,broken clouds
3,3,Katsuura,35.1333,140.3000,62.49,69,100,11.27,JP,light rain
4,4,Hobart,-42.8794,147.3294,60.03,67,100,2.30,AU,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
2,2,Itarema,-2.9248,-39.9167,77.70,88,67,5.77,BR,broken clouds
12,12,Hithadhoo,-0.6000,73.0833,82.99,75,29,10.40,MV,scattered clouds
17,17,Georgetown,5.4112,100.3354,84.13,93,20,6.91,MY,thunderstorm
18,18,Hilo,19.7297,-155.0900,76.28,89,75,6.91,US,light rain
24,24,Nanakuli,21.3906,-158.1547,76.10,73,100,9.22,US,overcast clouds
27,27,Poum,-20.2333,164.0167,76.01,75,100,2.73,NC,light rain
30,30,Bambous Virieux,-20.3428,57.7575,75.22,94,75,1.14,MU,thunderstorm with rain
33,33,Rikitea,-23.1203,-134.9692,78.48,63,2,10.27,PF,clear sky
35,35,Morant Bay,17.8815,-76.4093,75.70,83,6,6.06,JM,light rain
38,38,Puerto Ayora,-0.7393,-90.3518,75.16,95,100,7.20,EC,light rain


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                189
City                   189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Country                189
Current Description    189
dtype: int64

In [8]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [9]:
preferred_cities_df.notnull().sum()

City_ID                189
City                   189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Country                189
Current Description    189
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Itarema,BR,77.70,broken clouds,-2.9248,-39.9167,
12,Hithadhoo,MV,82.99,scattered clouds,-0.6000,73.0833,
17,Georgetown,MY,84.13,thunderstorm,5.4112,100.3354,
18,Hilo,US,76.28,light rain,19.7297,-155.0900,
24,Nanakuli,US,76.10,overcast clouds,21.3906,-158.1547,
27,Poum,NC,76.01,light rain,-20.2333,164.0167,
30,Bambous Virieux,MU,75.22,thunderstorm with rain,-20.3428,57.7575,
33,Rikitea,PF,78.48,clear sky,-23.1203,-134.9692,
35,Morant Bay,JM,75.70,light rain,17.8815,-76.4093,
38,Puerto Ayora,EC,75.16,light rain,-0.7393,-90.3518,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Itarema,BR,77.70,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
12,Hithadhoo,MV,82.99,scattered clouds,-0.6000,73.0833,Scoop Guest House
17,Georgetown,MY,84.13,thunderstorm,5.4112,100.3354,Cititel Penang
18,Hilo,US,76.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
24,Nanakuli,US,76.10,overcast clouds,21.3906,-158.1547,Camp Pālehua
27,Poum,NC,76.01,light rain,-20.2333,164.0167,
30,Bambous Virieux,MU,75.22,thunderstorm with rain,-20.3428,57.7575,Casa Tia Villa
33,Rikitea,PF,78.48,clear sky,-23.1203,-134.9692,People ThankYou
35,Morant Bay,JM,75.70,light rain,17.8815,-76.4093,Whispering Bamboo Cove Resort
38,Puerto Ayora,EC,75.16,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(27)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Itarema,BR,77.70,broken clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
12,Hithadhoo,MV,82.99,scattered clouds,-0.6000,73.0833,Scoop Guest House
17,Georgetown,MY,84.13,thunderstorm,5.4112,100.3354,Cititel Penang
18,Hilo,US,76.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
24,Nanakuli,US,76.10,overcast clouds,21.3906,-158.1547,Camp Pālehua
...,...,...,...,...,...,...,...
685,Anito,PH,83.19,overcast clouds,12.4497,125.2886,FELIPA LAODENIO PROPERTY
687,Kampot,KH,86.05,overcast clouds,10.6167,104.1833,Two Moons Hotel
689,Hambantota,LK,75.54,few clouds,6.1241,81.1185,Bungalow 63
691,Hays,US,78.94,few clouds,30.0502,-98.0336,Sinya on Lone Man Creek


In [15]:
preferred_cities_df.count()

City_ID                189
City                   189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Country                189
Current Description    189
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))